In [ ]:
from flask import Flask
from flask import request
from flask import render_template
        
app = Flask(__name__)

@app.route('/')
def my_form():
    return render_template("my_form.html")

@app.route('/', methods=['POST'])
def my_form_post():

     
    st=[]   
    data = request.form['q']
    print(data)
    
    if data == 'image captioning':
        import numpy as np
        from numpy import array
        import pandas as pd
        import pickle   
        import string
        import os
        import cv2
        from PIL import Image
        import glob
        from pickle import dump, load
        from time import time
        from keras.preprocessing import sequence
        from keras.models import Sequential
        from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                                Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
        from keras.optimizers import Adam, RMSprop
        from keras.layers.wrappers import Bidirectional
        from keras.layers.merge import add
        from keras.applications.inception_v3 import InceptionV3
        from keras.preprocessing import image
        from keras.models import Model
        from keras import Input, layers
        from keras import optimizers
        from keras.applications.inception_v3 import preprocess_input
        from keras.preprocessing.text import Tokenizer
        from keras.preprocessing.sequence import pad_sequences
        from keras.utils import to_categorical
        from keras.models import load_model
        from keras.preprocessing.image import load_img
        from keras.preprocessing.image import save_img
        from keras.preprocessing.image import img_to_array
        cam = cv2.VideoCapture(0) 
        while(cam.isOpened()): 
    
            ret,frame = cam.read() 
            if ret==True: 
          
                cv2.imwrite("C://Users//Abhilasha//Desktop//ppp.jpg",frame)
                cv2.imshow("frame",frame)
                break

        cam.release() 
        cv2.destroyAllWindows()

        image = frame

        def load_doc(filename):
            file = open(filename, 'r')
            text = file.read()
            file.close()
            return text

        filename = "C:\\Users\\Abhilasha\\image_captioning_flask\\Flickr_Data\\Flickr_TextData\\Flickr8k.token.txt"
        doc = load_doc(filename)
        def load_descriptions(doc):
            mapping = dict()
            for line in doc.split('\n'):
                tokens = line.split()
                if len(line) < 2:
                    continue
                image_id, image_desc = tokens[0], tokens[1:]
                image_id = image_id.split('.')[0]
                image_desc = ' '.join(image_desc)
                if image_id not in mapping:
                    mapping[image_id] = list()
                mapping[image_id].append(image_desc)
            return mapping

        descriptions = load_descriptions(doc)
        def clean_descriptions(descriptions):
            table = str.maketrans('', '', string.punctuation)
            for key, desc_list in descriptions.items():
                for i in range(len(desc_list)):
                    desc = desc_list[i]
                    desc = desc.split()
                    desc = [word.lower() for word in desc]
                    desc = [w.translate(table) for w in desc]
                    desc = [word for word in desc if len(word)>1]
                    desc = [word for word in desc if word.isalpha()]
                    desc_list[i] =  ' '.join(desc)

        clean_descriptions(descriptions)
        def to_vocabulary(descriptions):
            all_desc = set()
            for key in descriptions.keys():
                [all_desc.update(d.split()) for d in descriptions[key]]
            return all_desc

        vocabulary = to_vocabulary(descriptions)
        def save_descriptions(descriptions, filename):
            lines = list()
            for key, desc_list in descriptions.items():
                for desc in desc_list:
                    lines.append(key + ' ' + desc)
            data = '\n'.join(lines)
            file = open(filename, 'w')
            file.write(data)
            file.close()

        save_descriptions(descriptions, 'descriptions.txt')
        def load_set(filename):
            doc = load_doc(filename)
            dataset = list()
            for line in doc.split('\n'):
                if len(line) < 1:
                    continue
                identifier = line.split('.')[0]
                dataset.append(identifier)
            return set(dataset)

        filename = 'C:\\Users\\Abhilasha\\image_captioning_flask\\Flickr_Data\\Flickr_TextData\\Flickr_8k.trainImages.txt'
        train = load_set(filename)
        images = 'C:\\Users\\Abhilasha\\image_captioning_flask\\Flickr_Data\\Images\\'
        img = glob.glob(images + '*.jpg')
        train_images_file = 'C:\\Users\\Abhilasha\\image_captioning_flask\\Flickr_Data\\Flickr_TextData\\Flickr_8k.trainImages.txt'
        train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

        train_img = []

        for i in img:
            if i[len(images):] in train_images: 
                train_img.append(i) 
        test_images_file = 'C:\\Users\\Abhilasha\\image_captioning_flask\\Flickr_Data\\Flickr_TextData\\Flickr_8k.testImages.txt'
        test_images = set(open(test_images_file, 'r').read().strip().split('\n'))
        test_img = []

        for i in img: 
            if i[len(images):] in test_images: 
                test_img.append(i) 
        def load_clean_descriptions(filename, dataset):
            doc = load_doc(filename)
            descriptions = dict()
            for line in doc.split('\n'):
                tokens = line.split()
                image_id, image_desc = tokens[0], tokens[1:]
                if image_id in dataset:
                    if image_id not in descriptions:
                        descriptions[image_id] = list()
                    desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
                    descriptions[image_id].append(desc)
            return descriptions

        train_descriptions = load_clean_descriptions('descriptions.txt', train)
        def preprocess(image_path):
            img = load_img(image_path, target_size=(299, 299))
            x = img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            return x
        model = InceptionV3(weights='imagenet')
        model_new = Model(model.input, model.layers[-2].output)
        def encode(image):
            image = preprocess(image) 
            fea_vec = model_new.predict(image) 
            fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) 
            return fea_vec
        train_features = load(open("C:\\Users\\Abhilasha\\image_captioning_flask\\pickle\\encoded_train_images.pkl", "rb"))

        all_train_captions = []
        for key, val in train_descriptions.items():
            for cap in val:
                all_train_captions.append(cap)

        word_count_threshold = 10
        word_counts = {}
        nsents = 0
        for sent in all_train_captions:
            nsents += 1
            for w in sent.split(' '):
                word_counts[w] = word_counts.get(w, 0) + 1

        vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
        ixtoword = {}
        wordtoix = {}

        ix = 1
        for w in vocab:
            wordtoix[w] = ix
            ixtoword[ix] = w
            ix += 1
        vocab_size = len(ixtoword) + 1
        def to_lines(descriptions):
            all_desc = list()
            for key in descriptions.keys():
                [all_desc.append(d) for d in descriptions[key]]
            return all_desc

        def max_length(descriptions):
            lines = to_lines(descriptions)
            return max(len(d.split()) for d in lines)
        max_length = max_length(train_descriptions)
        def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
            X1, X2, y = list(), list(), list()
            n=0
            while 1:
                for key, desc_list in descriptions.items():
                    n+=1
                    photo = photos[key+'.jpg']
                    for desc in desc_list:
                        seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                        for i in range(1, len(seq)):
                            in_seq, out_seq = seq[:i], seq[i]
                            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                            X1.append(photo)
                            X2.append(in_seq)
                            y.append(out_seq)
                    if n==num_photos_per_batch:
                        yield [[array(X1), array(X2)], array(y)]
                        X1, X2, y = list(), list(), list()
                        n=0
        glove_dir = 'C:\\Users\\Abhilasha\\image_captioning_flask\\glove6b200d'
        embeddings_index = {} 
        f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()
    
        embedding_dim = 200
        embedding_matrix = np.zeros((vocab_size, embedding_dim))

        for word, i in wordtoix.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    
        model=load_model('C:\\Users\\Abhilasha\\image_captioning_flask\\model_weights\\model_39.h5')
        image_p = "C://Users//Abhilasha//Desktop//ppp.jpg"
   
        start = time()
        encoding_test = {}
        encoding_test[image_p[len(images):]] = encode(image_p)
        print("Time taken in seconds =", time()-start)
        with open("C:\\Users\\Abhilasha\\image_captioning_flask\\pickle\\encoded_test_image_single.pkl", "wb") as encoded_pickle_single:
            pickle.dump(encoding_test, encoded_pickle_single)
        with open("C:\\Users\\Abhilasha\\image_captioning_flask\\pickle\\encoded_test_image_single.pkl", "rb") as encoded_pickle_single:
            encoding_test_single = load(encoded_pickle_single)
        def greedySearch(photo):
            in_text = 'startseq'
            for i in range(max_length):
                sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
                sequence = pad_sequences([sequence], maxlen=max_length)
                yhat = model.predict([photo,sequence], verbose=0)
                yhat = np.argmax(yhat)
                word = ixtoword[yhat]
                in_text += ' ' + word
                if word == 'endseq':
                    break
            final = in_text.split()
            final = final[1:-1]
            final = ' '.join(final)
            return final
    
        for i in range (0,1):
            pic = list(encoding_test.keys())[i]
            image = encoding_test[pic].reshape((1,2048))

            ll=greedySearch(image)
   
            return render_template('index.html', message=ll)
    
    elif data == 'find faces':
        import os
        import face_recognition
        import cv2
        cam = cv2.VideoCapture(0) 
        while(cam.isOpened()): 
    
            ret,frame = cam.read() 
            if ret==True: 
          
                cv2.imwrite("C://Users//Abhilasha//Desktop//ppp.jpg",frame)
                cv2.imshow("frame",frame)
                break

        cam.release() 
        cv2.destroyAllWindows()
        img = frame

        faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        faces = faceCascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=3, minSize=(30, 30))

        print("[INFO] Found {0} Faces!".format(len(faces)))

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

        status = cv2.imwrite('C:\\Users\\Abhilasha\\faces_detected.jpg', img)
        print("[INFO] Image faces_detected.jpg written to filesystem: ", status)
        images = os.listdir('E://my_faces')

        for (x, y, w, h)  in faces:

            frame = img[y:y+h,x:x+w]

            cv2.imwrite("C://Users//Abhilasha//st.jpg", frame)
            image_to_be_matched = face_recognition.load_image_file("C://Users//Abhilasha//st.jpg")
            encodings = face_recognition.face_encodings(image_to_be_matched, num_jitters=100)
            if len(encodings) > 0:
                image_to_be_matched_encoded = encodings[0]
            else:
                continue

            for image in images:
   
                current_image = face_recognition.load_image_file("E://my_faces//" + image)
                current_image_encoded = face_recognition.face_encodings(current_image)[0]
                result = face_recognition.compare_faces([image_to_be_matched_encoded], current_image_encoded, tolerance = 0.5)
                if result[0] == True:
                    print("Matched: "+image)
                    st.append(image.split('.')[0])
  
        list_1_string = ' '.join(st)
        list_1_string = "Found " + list_1_string
    
        return render_template('index.html', message=list_1_string)
    
    elif data == 'text detection':
        import numpy as np
        import cv2
        import re
        from textblob import TextBlob
        from imutils.object_detection import non_max_suppression
        import pytesseract
        cam = cv2.VideoCapture(0) 
        while(cam.isOpened()): 
    
            ret,frame = cam.read() 
            if ret==True: 
          
                cv2.imwrite("C://Users//Abhilasha//Desktop//ppp.jpg",frame)
                cv2.imshow("frame",frame)
                break


        cam.release() 
        cv2.destroyAllWindows()

        image = frame
        

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        orig = gray.copy()
        (origH, origW) = image.shape[:2]
 
        (newW, newH) = (320,320)

        rW = origW / float(newW)
        rH = origH / float(newH)

        image = cv2.resize(image, (newW, newH))
        (H, W) = image.shape[:2]

        blob = cv2.dnn.blobFromImage(image, 1.0, (W, H), (123.68, 116.78, 103.94), swapRB=True, crop=False)
        net = cv2.dnn.readNet(r'C:\\Users\\Abhilasha\\final_hackabit1\\frozen_east_text_detection.pb')

        layerNames = [ "feature_fusion/Conv_7/Sigmoid","feature_fusion/concat_3"]
        net.setInput(blob)
        (scores, geometry) = net.forward(layerNames)
        def predictions(prob_score, geo):
            (numR, numC) = prob_score.shape[2:4]
            boxes = []
            confidence_val = []
            for y in range(0, numR):
                scoresData = prob_score[0, 0, y]
                x0 = geo[0, 0, y]
                x1 = geo[0, 1, y]
                x2 = geo[0, 2, y]
                x3 = geo[0, 3, y]
                anglesData = geo[0, 4, y]

                for i in range(0, numC):
                    if scoresData[i] < 0.9:
                        continue
                    (offX, offY) = (i * 4.0, y * 4.0)

                    angle = anglesData[i]
                    cos = np.cos(angle)
                    sin = np.sin(angle)
                    h = x0[i] + x2[i]
                    w = x1[i] + x3[i]
                    endX = int(offX + (cos * x1[i]) + (sin * x2[i]))
                    endY = int(offY - (sin * x1[i]) + (cos * x2[i]))
                    startX = int(endX - w)
                    startY = int(endY - h)

                    boxes.append((startX, startY, endX, endY))
                    confidence_val.append(scoresData[i])

            return (boxes, confidence_val)
        (boxes, confidence_val) = predictions(scores, geometry)
        boxes = non_max_suppression(np.array(boxes), probs=confidence_val)
        results = []

        for (startX, startY, endX, endY) in boxes:

            startX = int(startX * rW)
            startY = int(startY * rH)
            endX = int(endX * rW)
            endY = int(endY * rH)
            r = orig[startY:endY, startX:endX] 
            configuration = ("-l eng --oem 1 --psm 6")
 
            pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract'
            text = pytesseract.image_to_string(r, config=configuration)
            results.append(((startX, startY, endX, endY), text))
   
        orig_image = orig.copy()

        for ((start_X, start_Y, end_X, end_Y), text) in results:
            print("{}\n".format(text))
            st.append(format(text))

            text = "".join([x if ord(x) < 128 else "" for x in text]).strip()

        list_1_string = ' '.join(st)
        list_1_string = " ".join(re.findall(r"[a-zA-Z0-9]+", list_1_string))
        list_1_string = TextBlob(list_1_string.lower())
        return render_template('index.html', message=list_1_string.correct())
    
    else:
        message = " Sorry you selected a wrong option"
        print(data)
        return render_template('index.html', message=message)
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Oct/2019 02:23:03] "GET / HTTP/1.1" 200 -


find faces
[INFO] Found 1 Faces!
[INFO] Image faces_detected.jpg written to filesystem:  True
Matched: Abhilasha.jpg


127.0.0.1 - - [20/Oct/2019 02:25:07] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2019 02:26:47] "GET / HTTP/1.1" 200 -


find faces
[INFO] Found 2 Faces!
[INFO] Image faces_detected.jpg written to filesystem:  True
Matched: Abhilasha.jpg


127.0.0.1 - - [20/Oct/2019 08:15:28] "POST / HTTP/1.1" 200 -


Matched: Rashi.jpg
